# AIRLINE - CUSTOMER SATISFACTION

![5-star reviews icon](https://resource.gsx.com/5-star-reviews-icon.png)

### <span style="color:gray">**ÍNDICE**</span>

1) Abstract
2) Selección de APIs

### <span style="color:gray">**1. ABSTRACT**</span>

La premisa inicial consiste en la creación de un modelo destinado a anticipar las decisiones de compra de pasajeros de buses de larga distancia, basándose en su historial de comportamiento o del comportamiento de usuarios con atributos similares. Dada la ausencia de un conjunto de datos específico para este propósito, se utilizó información proveniente de una aerolínea como sustituto temporal, con la posibilidad de replicar el proceso en el eventual acceso a la información del servicio mencionado.

### <span style="color:gray">**2. SELECCIÓN DE APIS**</span>

Dado que el trabajo busca realizar un discovery de diferentes herramientas para poder luego aplicarlos a casos concretos en proyectos para una empresa de buses de larga distancia, se consiguieron dos APIs que podrían ayudar a pronosticar picos de demanda debido a feriados, fechas especiales y eventos como conciertos y recitales en diferentes partes de Argentina. 

Por esto es que se investigaron dos APIs: el calendario 2024 de feriados en Argentina y un calendario de conciertos y festivales venideros en Argentina.

Las APIs utilizadas son: 
- Calendario Feriados 2024: https://date.nager.at/Api
- Conciertos Jambase: https://apidocs.jambase.com/docs/jambase-api/branches/main/xdv7rgbi1x762-jam-base-concert-data-api

#### <span style="color:gray">**2.1. Antes de empezar**</span>

In [62]:
import pandas as pd
import numpy as np

from colorama import Fore, Style

import warnings

import json
import http.client
import requests

# Desactivar advertencias específicas
warnings.filterwarnings("ignore", category=UserWarning, module="seaborn")

# Formatos
colores = ['#0033a0', '#eb0029'] #azul y rojo
colores_satif = ['#eb0029', '#8DB600'] #rojo y verde manzana

def format_thousands(x, pos):
    return '{:,.0f}'.format(x).replace(',', '.')

#### <span style="color:gray">**2.2. API Calendario Feriados 2024**</span>

In [71]:
year = 2024 

holidays_url = f'https://date.nager.at/api/v2/PublicHolidays/{year}/ar'

df_holidays = requests.get(holidays_url)
print(df_holidays)

<Response [200]>


In [72]:
print(len(df_holidays.text))

texto = df_holidays.text

jsondata = json.loads(texto)
df_holidays = pd.DataFrame.from_dict(jsondata)

df_holidays.info()

df_holidays.head()

3194
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         16 non-null     object 
 1   localName    16 non-null     object 
 2   name         16 non-null     object 
 3   countryCode  16 non-null     object 
 4   fixed        16 non-null     bool   
 5   global       16 non-null     bool   
 6   counties     0 non-null      object 
 7   launchYear   1 non-null      float64
 8   type         16 non-null     object 
dtypes: bool(2), float64(1), object(6)
memory usage: 1.0+ KB


,date,localName,name,countryCode,fixed,global,counties,launchYear,type
0,2024-01-01,Año Nuevo,New Year's Day,AR,True,True,None,NaN,Public
1,2024-02-12,Carnaval,Carnival,AR,False,True,None,NaN,Public
2,2024-02-13,Carnaval,Carnival,AR,False,True,None,NaN,Public
3,2024-03-24,Día Nacional de la Memoria por la Verdad y la ...,Day of Remembrance for Truth and Justice,AR,True,True,None,NaN,Public
4,2024-03-29,Viernes Santo,Good Friday,AR,False,True,None,NaN,Public


#### <span style="color:gray">**2.3. API Jambase Concerts**</span>

In [74]:
conn = http.client.HTTPSConnection("www.jambase.com")

headers = { 'Accept': "application/json" }

api_key = 'MY_API_KEY'

conn.request("GET", f"/jb-api/v1/events?geoCountryIso2=AR&geoCountryIso3=ARG&apikey={api_key}", headers=headers)

res = conn.getresponse()
data = res.read()

json_data = json.loads(data.decode("utf-8"))

df_concerts = pd.json_normalize(json_data.get('events', []))
df_concerts.info()
df_concerts.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 44 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   @type                                          30 non-null     object 
 1   name                                           30 non-null     object 
 2   identifier                                     30 non-null     object 
 3   url                                            30 non-null     object 
 4   image                                          30 non-null     object 
 5   sameAs                                         30 non-null     object 
 6   datePublished                                  30 non-null     object 
 7   dateModified                                   30 non-null     object 
 8   eventStatus                                    30 non-null     object 
 9   startDate                                      30 non-nu

,@type,name,identifier,url,image,sameAs,datePublished,dateModified,eventStatus,startDate,...,location.address.addressCountry.identifier,location.address.addressCountry.name,location.address.addressCountry.alternateName,location.address.x-streetAddress2,location.address.x-timezone,location.address.x-jamBaseMetroId,location.address.x-jamBaseCityId,location.geo.@type,location.geo.latitude,location.geo.longitude
0,Concert,FISHER at Mute Club De Mar,jambase:11389953,https://www.jambase.com/show/fisher-mute-club-...,https://www.jambase.com/wp-content/uploads/202...,[],2023-12-07T17:14:48Z,2023-12-20T07:01:39Z,scheduled,2024-01-04T19:00:00,...,AR,Argentina,ARG,,America/Argentina/Buenos_Aires,,,GeoCoordinates,-38.0959,-57.5536
1,Concert,David Guetta at Movistar Arena,jambase:11292475,https://www.jambase.com/show/david-guetta-movi...,https://www.jambase.com/wp-content/uploads/202...,[],2023-11-15T14:57:22Z,2023-12-20T06:17:31Z,scheduled,2024-01-06T21:00:00,...,AR,Argentina,ARG,,America/Argentina/Buenos_Aires,,150433,GeoCoordinates,-34.5946,-58.4477
2,Concert,Cassian at La Estación Córdoba,jambase:11215625,https://www.jambase.com/show/cassian-la-estaci...,https://www.jambase.com/wp-content/uploads/202...,[],2023-10-27T16:08:03Z,2023-12-20T06:28:34Z,scheduled,2024-01-12T19:00:00,...,AR,Argentina,ARG,,America/Argentina/Cordoba,,,GeoCoordinates,-31.3410,-64.4607
3,Concert,Santa Fe Klan at Luna Park,jambase:10633515,https://www.jambase.com/show/santa-fe-klan-lun...,https://www.jambase.com/wp-content/uploads/202...,[],2023-06-20T19:50:39Z,2023-12-20T06:17:31Z,scheduled,2024-02-03T20:30:00,...,AR,Argentina,ARG,,America/Argentina/Buenos_Aires,,,GeoCoordinates,-34.6020,-58.3684
4,Concert,Blanco White at La Trastienda,jambase:11186819,https://www.jambase.com/show/blanco-white-la-t...,https://www.jambase.com/wp-content/uploads/201...,[],2023-10-20T20:06:13Z,2023-12-20T06:17:31Z,scheduled,2024-02-08T19:30:00,...,AR,Argentina,ARG,,America/Argentina/Catamarca,,,GeoCoordinates,-34.6131,-58.3705
